In [2]:
import numpy as np
from scipy.optimize import linprog
from functools import reduce
from itertools import product, permutations
from scipy.special import comb

In [5]:
## single-qutrit permutation matrices
P = {'P0': np.array([[1,0,0],
                                         [0,1,0],
                                         [0,0,1]]),
                         'P1': np.array([[1,0,0],
                                         [0,0,1],
                                         [0,1,0]]),
                         'P2': np.array([[0,1,0],
                                         [1,0,0],
                                         [0,0,1]]),
                         'P3': np.array([[0,1,0],
                                         [0,0,1],
                                         [1,0,0]]),
                         'P4': np.array([[0,0,1],
                                         [1,0,0],
                                         [0,1,0]]),
                         'P5': np.array([[0,0,1],
                                         [0,1,0],
                                         [1,0,0]]),
                        }

n_permutations = 6

In [7]:
Hamiltonian = np.diag([0,0,0,0,11,21,0,12,22])

In [344]:
p_q2 = 4
p_q1 = 5

pmat = np.kron(P['P{}'.format(p_q1)], P['P{}'.format(p_q2)])

H = reduce(np.dot, [pmat, Hamiltonian, pmat.T])

In [346]:
P['P5']

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])

In [345]:
print(np.diag(H)[6:])
print(np.diag(H)[3:6])
print(np.diag(H)[0:3])

[0 0 0]
[21  0 11]
[22  0 12]


In [6]:
# set up the linear programming problem
alpha_dict_01 = {'11': 2*np.pi*(-0.27935),
             '12': 2*np.pi*(0.1599),
             '21': 2*np.pi*(-0.52793),
             '22': 2*np.pi*(-0.742967)}



alpha_dict_test = {
                '00': 0,
                '01': 0,
                '02': 0,
                '10': 0,
                '20': 0,
                '11': 11,
                '12': 12,
                '21': 21,
                '22': 22
}


def generate_equality_matrix(permutations_to_use, alpha_dict):
    
    a00 = alpha_dict.get('00',0)
    a01 = alpha_dict.get('01',0)
    a02 = alpha_dict.get('02',0)    
    a10 = alpha_dict.get('10',0)
    a11 = alpha_dict.get('11',0)
    a12 = alpha_dict.get('12',0)
    a20 = alpha_dict.get('20',0)    
    a21 = alpha_dict.get('21',0)    
    a22 = alpha_dict.get('22',0)
    
    base_hamiltonian = np.array([a00,a01,a02,a10,a11,a12,a20,a21,a22])
    
    equality_matrix_full = np.zeros((len(permutations_to_use), 9)) # has the local phases too
    
    for ind, permutation in enumerate(permutations_to_use):
        equality_matrix_full[ind] = np.dot(permutation, base_hamiltonian)
    
    # now subtract the z rotation phases
    equality_matrix_z1 = np.array([equality_matrix_full[:,i] - equality_matrix_full[:,3*(i//3)] for i in range(9)]).T
    equality_matrix_z2 = np.array([equality_matrix_z1[:,i] - equality_matrix_z1[:,i%3] for i in range(9)]).T
    
    return equality_matrix_z2[:,[4,5,7,8]].T

def generate_obj_function(permutations_to_use):
    return np.ones(len(permutations_to_use))

def generate_equality_vector(desired_phases, m_vector):
    return desired_phases + 2*np.pi*np.array(m_vector)

In [7]:
all_1q_perms = ['P{}'.format(a) for a in range(6)]
all_permutations_list = list(product(all_1q_perms, all_1q_perms))
all_permutations = [np.kron(P[a[0]], P[a[1]]) for a in all_permutations_list]

permutations_initial_list = [('P0','P0'), ('P1','P1'),('P2','P2'), ('P3','P3'), ('P4','P4'), ('P5','P5')]
permutations_initial = [np.kron(P[a[0]], P[a[1]]) for a in permutations_initial_list]

permutations_original_list = [('P0','P0'), ('P0','P1'),('P1','P0'), ('P1','P1')]
permutations_original = [np.kron(P[a[0]], P[a[1]]) for a in permutations_original_list]


In [8]:
t1 = generate_equality_matrix(all_permutations, alpha_dict_01)
t2 = generate_equality_matrix(permutations_original, alpha_dict_01)

In [9]:
c1 = generate_obj_function(all_permutations)
c2 = generate_obj_function(permutations_original)

In [10]:
desired_phases = np.array([2*np.pi/3, 4*np.pi/3, 4*np.pi/3, 2*np.pi/3])

In [51]:
for m1 in range(-1,1):
    for m2 in range(-1,1):
        for m3 in range(-1,1):
            for m4 in range(-1,1):
                b_eq = generate_equality_vector(desired_phases, [m1,m2,m3,m4])
                result = linprog(c2, A_eq = t2, b_eq = b_eq)
                if result.success:
                    if result.fun < 2:
                        print(m1,m2,m3,m4)
                        print(result.fun)

-1 -1 -1 -1
1.4384898158517263


In [14]:
alpha_dict_01.values()

dict_values([-1.7552078155606172, 1.0046813306180158, -3.317082019219319, -4.668199338119296])

In [29]:
# does this correspond with majorization?

sorted_alphas = list(reversed(sorted(-1*np.array(list(alpha_dict_01.values())))))
sorted_desired_phases = list(reversed(sorted(desired_phases)))

In [30]:
sorted_desired_phases

[4.1887902047863905,
 4.1887902047863905,
 2.0943951023931953,
 2.0943951023931953]

In [31]:
sorted_alphas

[4.668199338119296, 3.317082019219319, 1.7552078155606172, -1.0046813306180158]

In [48]:
def minimum_time_to_majorize(l1, l2):
    """ determines the minimum time, t, at which list2 * t majorizes list1"""
    
    # sort list1 in descending order 
    l1_sorted = np.array(list(reversed(sorted(l1))))
    l1_partial_sums = np.array([sum(l1_sorted[:i+1]) for i in range(len(l1_sorted))])
    
    # sort list2 in descending order
    l2_sorted = np.array(list(reversed(sorted(l2))))
    l2_partial_sums = np.array([sum(l2_sorted[:i+1]) for i in range(len(l2_sorted))])
    
    return np.max(l1_partial_sums/l2_partial_sums)

In [55]:
for m1 in range(-2,2):
    for m2 in range(-2,2):
        for m3 in range(-2,2):
            for m4 in range(-2,2):
                print(minimum_time_to_majorize(sorted_desired_phases + np.pi*np.array([m1,m2,m3,m4]), sorted_alphas))

-0.4486516000486339
-0.22432580002431696
0.4486516000486338
1.1216290001215845
-0.22432580002431696
-0.22432580002431696
0.4486516000486338
1.1216290001215845
0.4486516000486338
0.4486516000486338
0.5245638841437714
1.1216290001215845
1.1216290001215845
1.1216290001215845
1.1216290001215845
1.3114097103594284
0.22432580002431685
0.22432580002431685
0.4486516000486338
1.1216290001215845
0.22432580002431685
0.22432580002431685
0.4486516000486338
1.1216290001215845
0.4486516000486338
0.4486516000486338
0.5375487476081768
1.1216290001215845
1.1216290001215845
1.1216290001215845
1.1216290001215845
1.3114097103594284
0.8973032000972676
0.8973032000972676
0.8973032000972676
1.1802687393234856
0.8973032000972676
0.8973032000972676
0.8973032000972676
1.1802687393234856
0.8973032000972676
0.8973032000972676
0.8973032000972676
1.1826072447379892
1.1802687393234856
1.1802687393234856
1.1826072447379892
1.505136493302895
1.5702806001702183
1.5702806001702183
1.5702806001702183
1.5736916524313143
1.

In [320]:
for m1 in range(-1,1):
    for m2 in range(-1,1):
        for m3 in range(-1,1):
            for m4 in range(-1,1):
                b_eq = generate_equality_vector(desired_phases, [m1,m2,m3,m4])
                result = linprog(c1, A_eq = t1, b_eq = b_eq)
                if result.success:
                    if result.fun < 1.1:
                        print(m1,m2,m3,m4)
                        print(result.x)
                        print(result.fun)

-1 -1 -1 0
[0.         0.         0.         0.         0.         0.3986566
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.13608932 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.13608932 0.         0.         0.
 0.         0.         0.         0.         0.3986566  0.        ]
1.0694918363014405
0 -1 -1 -1
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.3986566  0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.13608932 0.         0.         0.
 0.         0.         0.         0.         0.         0.3986566
 0.         0.         0.         0.13608932 0.         0.        ]
1.0694918363014405
0 0 0 0
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.3986566  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0. 

In [266]:
def permutation_transitions(P1, P2):
    lookup_table = np.array([[0,1,1,2,2,3],
                            [1,0,2,3,1,2],
                            [1,2,0,1,3,2],
                            [2,3,1,0,2,1],
                            [2,1,3,2,0,1],
                            [3,2,2,1,1,0]])
    return lookup_table[P1,P2]

def total_permutation_transitions(list_of_permutations):
    # determines the total number of transitions to go through the given list of permutations
    return sum([permutation_transitions(list_of_permutations[i], list_of_permutations[i+1]) for i in range(len(list_of_permutations)-1)])

In [271]:
def fewest_singlequbit_gates(list_of_permutations):
    # module should do import:
    #     from itertools import permutations
    
    min_trans = np.inf
    min_order = None
    for permutation_order in permutations(list_of_permutations):
        permutation_order_q1 = [0] + [a[0] for a in permutation_order] + [0] 
        permutation_order_q2 = [0] + [a[1] for a in permutation_order] + [0] 
        
        total_transitions = total_permutation_transitions(permutation_order_q1) + total_permutation_transitions(permutation_order_q2)
        if total_transitions < min_trans:
            min_trans = total_transitions
            min_order = permutation_order
    
    return min_trans, min_order

In [274]:
list1 = [(0,5), (2,3), (4,2), (5,4)]
list2 = [(1,4), (3,2), (4,5), (5,3)]
list3 = [(1,3), (3,5), (4,1), (5,0)]

In [273]:
fewest_singlequbit_gates(list1)

(16, ((0, 5), (2, 3), (5, 4), (4, 2)))

In [275]:
fewest_singlequbit_gates(list2)

(12, ((1, 4), (4, 5), (5, 3), (3, 2)))

In [276]:
fewest_singlequbit_gates(list3)

(16, ((1, 3), (3, 5), (4, 1), (5, 0)))

In [334]:
def ver1gate_map(alpha_dict):
    a11 = alpha_dict.get('11',0)
    a12 = alpha_dict.get('12',0)
    a21 = alpha_dict.get('21',0)    
    a22 = alpha_dict.get('22',0)
    
    return np.array([[-a22, a22-a21, a12 - a11 - a22 + a21, a11 - a21],
                     [a21-a22, a22, a21 - a11, a22 - a21 - a12 + a11],
                     [-a12, a11 - a12 - a21 + a22, a21 - a22, a11],
                     [a11-a12, a22-a12, a21, a11 - a12],
                    ])

In [341]:
gate_times = np.linalg.solve(ver1gate_map(alpha_dict_01), desired_phases - 2*np.pi*np.array([0,1,1,1]))

In [343]:
gate_times

array([0.3986566 , 0.3986566 , 0.13608932, 0.13608932])